In [6]:
from selenium import webdriver
import regex as re
import pandas as pd
import numpy as np
zhouyiguaxu=['乾','坤','屯','蒙','需','訟','師','比','小畜','履','泰','否','同人','大有','謙','豫','隨','蠱','臨','觀','噬嗑','賁','剝','復','無妄','大畜','頤','大過','坎','離','咸','恆','遯','大壯','晉','明夷','家人','睽','蹇','解','損','益','夬','姤','萃','升','困','井','革','鼎','震','艮','漸','歸妹','豐','旅','巽','兌','渙','節','中孚','小過','既濟','未濟']
w1=['4103', '4105', '4106', '4107', '4108', '4109', '4110', '4111', '4112', '4113', '4126', '4127', '4140', '4141', '4142', '4143', '4144', '4145', '4146', '4147', '4148', '4149', '4150', '4152', '4153', '4159', '4164', '4167', '4168', '4169', '4170', '4171', '4172', '4173', '4174', '4175', '4176', '4177', '4179', '4180', '4181', '4182', '4183', '4184', '4185', '4186', '4187', '4188', '4189', '4190', '4192', '4193', '4194', '4195', '4196', '4197', '4198', '4200', '4212', '4244', '4255', '4256', '4257', '4263']

In [4]:
driver = webdriver.Chrome()
L=[]
for i in range(64):
    url='https://m.zhouyi.cc/zhouyi/yijing64/'+w1[i]+'.html'
    driver.get(url)
    a=driver.find_elements_by_xpath('//*[@class="tip_text"]') 
    a1=[]
    for j in range(7):
        a1.append(a[j].text)
    L.append(a1)

In [24]:
duisi1='''九四。商兑，未宁，介疾有喜。
象曰：九四之喜，有庆也。'''
duisi2='''九四：商谈恢复邦交之事，尚未达成协议，但两国的矛盾分歧有了愈合的趋势。
《象辞》说：九四爻辞所讲的喜，即是指将有喜庆之事。'''
duisi3='得此爻者，从商获利，或进人口，不良者或有疾病，谋望不成。做官的会身居要职，升迁有望。'
duisi4='奋斗将成，斟酌行止。'
duisi5='忧心之事，商量解决。'
duisi6='多疾不安；再三说媒而成。'
duisi7='心神不安，喜事舒怀。'
duisi8='九四爻动变得周易第60卦：水泽节。这个卦是异卦（下兑上坎）相叠。兑为泽，坎为水。泽有水而流有限，多必溢于泽外。因此要有节度，故称节。节卦与涣卦相反，互为综卦，交相使用。天地有节度才能常新，国家有节度才能安稳，个人有节度才能完美。'

In [29]:
L2=[]
for i in range(len(L)):
    for j in range(7):
        m=[s.strip() for s in re.split("..原文\n|..爻辞\n|白话文解释\n|《断易天机》解|北宋易学家邵雍解|传统解卦|台湾国学大儒傅佩荣解|..变卦\n|.+哲学含义", L[i][j])]
        if j==0:
            weidu=[s.strip() for s in re.split("\n",m[5])]
            if len(m)>7:
                d={'原文':m[1],'白话文':m[2],'《断易天机》':m[3],'邵雍':m[4],'时运':weidu[0][3:],'财运':weidu[1][3:],'家宅':weidu[2][3:],'身体':weidu[3][3:],'传统':m[6],'哲学含义':m[7].strip('：')}
            else:
                d={'原文':m[1],'白话文':m[2],'《断易天机》':m[3],'邵雍':m[4],'时运':weidu[0][3:],'财运':weidu[1][3:],'家宅':weidu[2][3:],'身体':weidu[3][3:],'传统':m[6]}
        else:
            if i==57 and j==4: 
                d={'吉凶':'平','原文':duisi1,'白话文':duisi2,'邵雍':duisi3,'时运':duisi4,'财运':duisi5,'家宅':duisi6,'身体':duisi7,'变卦':duisi8}
                L2.append(d)
                continue
            weidu=[s.strip() for s in re.split("\n",m[4])]
            if len(m)>6:
                d={'吉凶':m[3][0],'原文':m[1],'白话文':m[2],'邵雍':m[3][2:],'时运':weidu[0][3:],'财运':weidu[1][3:],'家宅':weidu[2][3:],'身体':weidu[3][3:],'变卦':m[5],'哲学含义':m[6].strip('：\n')}
            else:
                d={'吉凶':m[3][0],'原文':m[1],'白话文':m[2],'邵雍':m[3],'时运':weidu[0][3:],'财运':weidu[1][3:],'家宅':weidu[2][3:],'身体':weidu[3][3:],'变卦':m[5]}
        L2.append(d)
        
content=['卦','一爻','二爻','三爻','四爻','五爻','六爻']
Index=[]
for i in range(len(L)):
    for j in range(7):
        Index.append(zhouyiguaxu[i]+content[j])
        
df=pd.DataFrame(L2)
df.index=Index



In [30]:
value={'吉':1,'中吉':0.75,'平':0.5,'中和':0.5,'吉凶參半':0.5,'困':0.5,'中兇':0.25,'凶':0,'兇':0}

df['顺序']=range(len(df))


df2=pd.DataFrame(pd.read_excel('卦2.xlsx'))
df2=df2.set_index('卦名')

df0=pd.merge(df, df2, left_index=True, right_index=True,how='outer')
df0=df0.fillna('')

df0["运势"] = df0["吉凶_x"] + df0["吉凶_y"]
df0=df0.sort_values(by=['顺序'])
df0=df0.drop(columns=['吉凶_x','吉凶_y','卦序','数值'])
df0.groupby('运势').count()
df0['数值']=[value[df0.iloc[i,13]]for i in range(len(df))]

df0.to_excel('六十四卦.xlsx')

In [31]:
df0

,原文,白话文,《断易天机》,邵雍,时运,财运,家宅,身体,传统,哲学含义,变卦,顺序,内容,运势,数值
乾卦,乾。元，亨，利，贞。\n象曰：天行健，君子以自强不息。,乾卦：大吉大利，吉利的贞卜。\n《象辞》说：天道刚健，运行不已。君子观此卦象，从而以天为法，...,乾象征天，六阳爻构成乾卦，为《易经》六十四卦之首。纯阳刚建，其性刚强，其行劲健，大通而至正，...,刚健旺盛，发育之功；完事顺利，谨防太强。\n得此卦者，天行刚健，自强不息，名利双收之象，宜把...,临事刚健，自强不息。,施比受有福，不利买而利卖。,积善有余庆；女子过刚宜慎重。,保健有恒。,这个卦是同卦（下乾上乾）相叠。象征天，喻龙（德才的君子），又象征纯粹的阳和健，表明兴盛强健。...,乾卦所包含的范围是：凡是积极向上的、刚健有力的、权威的、圆形的、男性长辈、珍贵的、富有的、寒...,,0,乾卦，陽剛，剛健，自強不息。乾六爻皆盈滴，故肥園，園滿、亭通，成功、重大。但剛多易折，含欠安...,吉,1.0
乾一爻,初九。潜龙勿用。\n象曰：潜龙勿用，阳在下也。,初九：潜藏的龙，无法施展。\n《象辞》说：潜藏的龙，无法施展，因为初九阳爻处在一卦的下位，所...,,得此爻者，宜沉稳待机，若一动作即生灾疾，谋事不利，谨防小人。做官的会有阻力。经商的会有阻滞。...,培养实力，等待机会。,宜守不宜攻。,娶妻小心。,保养为宜。,,乾卦第一爻爻辞释义：初九：潜龙勿用。经文意思是：身居下位，时机还没有成熟，所以应当像潜藏的龙...,初九爻动变得周易第44卦：天风姤。这个卦是异卦（下巽上乾）相叠。乾为天，巽为风。天下有风，吹...,1,,平,0.5
乾二爻,九二。见龙在田，利见大人。\n象曰：见龙在田，德施普也。,九二：龙出现在大地上，有利于会见贵族王公。\n《象辞》说：龙出现在大地上，喻指君子走出了压抑...,,得此爻者，会得到贵人的提拔。做官的会遇到明主，身居要职。读书人考试能获得佳绩。经商者获利。女...,得贵人之助，有发展机会。,开始涨价，官方采购。,婚姻大吉。,运动健身。,,九二：见龙在田，利见大人。爻辞释义：龙出现在田野之上，有利于见到大人物来发展自己。\n见：在...,九二爻动变得周易第13卦：天火同人。这个卦是异卦（下离上乾）相叠，乾为天，为君；离为火，为臣...,2,,吉,1.0
乾三爻,九三。君子终日乾乾，夕惕若；厉，无咎。\n象曰：终日乾乾，反复道也。,九三：有才德的君子始终是白天勤奋努力，夜晚戒惧反省，虽然处境艰难，但终究没有灾难。\n《象辞...,,得此爻者，会往来不停，财利难获，凡事不认真计划，躁动者会有损失。做官的会身兼重职，且事多繁冗...,功名未显，戒慎免咎。,日夜防范，可脱险获利。,勤俭保家；不宜攀结高亲。,小心保养。,,爻辞释义：乾乾：本是指太阳的运行不止，在这里比喻君子勤勉不懈。惕：有敬的意思，小心谨慎、警锡...,九三爻动变得周易第10卦：天泽履。这个卦是异卦（下兑上乾）相叠，乾为天，兑为泽，以天喻君，以...,3,,平,0.5
乾四爻,九四。或跃在渊，无咎。\n象曰：或跃在渊，进无咎也。,九四：龙也许跳进深潭，没有灾难。\n《象辞》说：龙也许跳进深潭，表示可以有所作为而没有灾难。,,得此爻者，百为艰难。做官的会停缺待职，等待机会。女命逢之，则安乐富贵也。,一举成名。,物价高涨，可保无咎。,有一时振兴之象。,继续健身。,,爻辞释义：本爻辞的意思是：龙或跃上天空，或停留在深渊，只要根据形势的需要而前进，就不会有错误...,九四爻动变得周易第9卦：风天小畜。这个卦是异卦（下乾上巽）相叠，乾为天，巽为风。喻风调雨顺，...,4,,平,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
未濟二爻,九二。曳其轮，贞吉。\n象曰：九二贞吉，中以行正也。,九二：提着腰带涉水过河。卜问得吉兆。\n《象辞》说：九二爻辞讲贞吉，因为九二阳爻处下卦中位，...,,得此爻者，安份守纪者，谋望遂意。做官的克服困难者，晋升有望。,上下相得，无往不利。,用心经营，满载而归。,美善之家；必得贤妇。,胸腹阻滞。,,未济卦第二爻，爻辞：九二：曳其轮，贞吉。\n爻辞释义曳：指拖住。\n本爻辞的意思是：拖住车轮...,九二爻动变得周易第35卦：火地晋。这个卦是异卦（下坤上离）相叠。离为日，为光明；坤为地。太阳...,443,,吉,1.0
未濟三爻,六三。未济，征凶，利涉大川。\n象曰：未济征凶，位不当也。,六三：渡不了河，出行有凶险。不利于涉水渡河。\n《象辞》说：渡不了河，出行有凶险，因为六三阴...,,得此爻者，宜见机行事，不宜妄动。做官的有因人成事之美。在商旅则涉川历险而利可获。,见机而作，妄动则凶。,把握良机，船运有利。,安居为宜；迎娶为吉。,预防祸祟，渡川以解。,,未济卦第三爻，爻辞：六三：未济，征凶，利涉大川。爻辞释义\n征：指征伐，进攻他人。\n本爻辞...,六三爻动变得周易第50卦：火风鼎。这个卦是异卦（下巽上离）相叠。燃木煮食，化生为熟，除旧布新...,444,,凶,0.0
未濟四爻,九四。贞吉，悔亡，震用伐鬼方。三年有赏于大国。\n象曰：贞吉悔亡，志行也。,九四：吉利的贞卜，没有悔恨。大动干戈，助殷讨伐鬼方，费时三年，打败了它，获得殷国的奖赏。\n...,,得此爻者，会得好人提举，获利丰厚。做官的会受到重用，名利双全。读书人会取得佳绩。,志得意满，名利双全。,远方经营，利润可观。,安居乐业；二年成婚。,可保三年。,,未济卦第四爻，爻辞：九四：贞吉，悔亡；震用伐鬼方，三年有赏于大国。爻辞释义\n震用：指声势浩...,九四爻动变得周易第4卦：山水蒙。这个卦是异卦（下坎上艮）相叠，艮是山的形象，喻止；坎是水的形...,445,,吉,1.0
未濟五爻,六五。贞吉，无悔，君子之光。有孚，吉。\n象曰：君子之光，其晖吉也。,六五：吉利的贞卜，没有悔恨。打了胜仗，捕获了俘虏，这是君子的光荣，吉利。\n《象辞》说：君子...,,得此爻者，谋望有成，财利丰厚。做官的能得到升迁。,运势正旺，诸事皆吉。,官方业务，获利稳定。,大吉之家；婚姻必成。,神清气爽。,,未济卦第五爻，爻辞：六五：贞吉，无悔；君子之光，有孚，吉。爻辞释义\n本爻辞的意思是：坚守正...,六五爻动变得周易第6卦：天水讼。这个卦是异卦（下坎上乾）相叠。同需卦相反，互为“综卦”。乾为...,446,,吉,1.0
